In [17]:
# Generieren aller Feiertage durch Claude und manuelle Überprüfung. Reformationstag war erstmals am 31.10.2017 gesetzlicher 
# Feiertag. Zeitraum an umsatzdaten orientiert, also 01.07.2013 bis 31.12.2017. Oster- und Pfingstsonntag fallen nicht in 
# die gestzlichen Feiertag, weil Sonntage, sind aber trotzdem aufgeführt.

import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict

def ostern(jahr: int) -> datetime:
    """Berechnet das Osterdatum für ein gegebenes Jahr (Gauß-Formel)"""
    k = jahr // 100
    m = 15 + (3 * k + 3) // 4 - (8 * k + 13) // 25
    s = 2 - (3 * k + 3) // 4
    a = jahr % 19
    d = (19 * a + m) % 30
    r = (d + a // 11) // 29
    og = 21 + d - r
    sz = 7 - (jahr + jahr // 4 + s) % 7
    oe = 7 - (og - sz) % 7
    os = og + oe
    
    if os > 31:
        return datetime(jahr, 4, os-31)
    return datetime(jahr, 3, os)

def get_feiertage(jahr: int) -> List[Dict]:
    """Erstellt eine Liste aller Feiertage in Schleswig-Holstein für ein Jahr"""
    oster_datum = ostern(jahr)
    
    # Basis-Feiertage ohne Reformationstag
    feiertage = [
        {"Datum": datetime(jahr, 1, 1), "Feiertag": "Neujahr"},
        {"Datum": oster_datum - timedelta(days=2), "Feiertag": "Karfreitag"},
        {"Datum": oster_datum, "Feiertag": "Ostersonntag"},
        {"Datum": oster_datum + timedelta(days=1), "Feiertag": "Ostermontag"},
        {"Datum": datetime(jahr, 5, 1), "Feiertag": "Tag_der_Arbeit"},
        {"Datum": oster_datum + timedelta(days=39), "Feiertag": "Christi_Himmelfahrt"},
        {"Datum": oster_datum + timedelta(days=49), "Feiertag": "Pfingstsonntag"},
        {"Datum": oster_datum + timedelta(days=50), "Feiertag": "Pfingstmontag"},
        {"Datum": datetime(jahr, 10, 3), "Feiertag": "Tag_der_Deutschen_Einheit"},
        {"Datum": datetime(jahr, 12, 25), "Feiertag": "Erster_Weihnachtstag"},
        {"Datum": datetime(jahr, 12, 26), "Feiertag": "Zweiter_Weihnachtstag"}
    ]
    
    # Füge Reformationstag ab 2017 hinzu
    if (jahr >= 2017):
        feiertage.append({"Datum": datetime(jahr, 10, 31), "Feiertag": "Reformationstag"})
    
    return feiertage

# Erstelle DataFrame mit allen Tagen
start_date = datetime(2013, 7, 1)
end_date = datetime(2019, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
df_dates = pd.DataFrame({'Datum': date_range})

# Sammle alle Feiertage
alle_feiertage = []
for jahr in range(2013, 2020):
    alle_feiertage.extend(get_feiertage(jahr))

# Konvertiere zu DataFrame
df_feiertage = pd.DataFrame(alle_feiertage)

# Definiere die Reihenfolge der Feiertage
feiertags_reihenfolge = [
    'Neujahr',
    'Karfreitag',
    'Ostersonntag',
    'Ostermontag',
    'Tag_der_Arbeit',
    'Christi_Himmelfahrt',
    'Pfingstsonntag',
    'Pfingstmontag',
    'Tag_der_Deutschen_Einheit',
    'Reformationstag',
    'Erster_Weihnachtstag',
    'Zweiter_Weihnachtstag'
]

# Initialisiere alle Feiertagsspalten mit 0
for feiertag in feiertags_reihenfolge:
    df_dates[feiertag] = 0

# Setze 1 für jeden Feiertag am entsprechenden Datum
for _, row in df_feiertage.iterrows():
    df_dates.loc[df_dates['Datum'] == row['Datum'], row['Feiertag']] = 1

# Formatiere das Datum als String
df_dates['Datum'] = df_dates['Datum'].dt.strftime('%d.%m.%Y')

# Ordne die Spalten in der gewünschten Reihenfolge
df_dates = df_dates[['Datum'] + feiertags_reihenfolge]

# Speichere als CSV
df_dates.to_csv('feiertage_sh_binary_2013_2019.csv', index=False, sep=';', encoding='utf-8')

# Zeige Beispiele zur Überprüfung
print("Erste Zeilen:")
print(df_dates.head())
print("\nSpaltenreihenfolge:")
print(df_dates.columns.tolist())
print("\nAnzahl der Feiertage pro Kategorie:")
print(df_dates.iloc[:, 1:].sum())

Erste Zeilen:
        Datum  Neujahr  Karfreitag  Ostersonntag  Ostermontag  Tag_der_Arbeit  \
0  01.07.2013        0           0             0            0               0   
1  02.07.2013        0           0             0            0               0   
2  03.07.2013        0           0             0            0               0   
3  04.07.2013        0           0             0            0               0   
4  05.07.2013        0           0             0            0               0   

   Christi_Himmelfahrt  Pfingstsonntag  Pfingstmontag  \
0                    0               0              0   
1                    0               0              0   
2                    0               0              0   
3                    0               0              0   
4                    0               0              0   

   Tag_der_Deutschen_Einheit  Reformationstag  Erster_Weihnachtstag  \
0                          0                0                     0   
1             

In [18]:
# Überprüfen der generierten Daten auf Korrektheit

# 1. Grundlegende Statistiken
print("Anzahl Tage pro Jahr:")
yearly_days = df_dates.groupby(df_dates['Datum'].str[-4:]).size()
print(yearly_days)

print("\nAnzahl Feiertage pro Jahr:")
# Summiere alle Feiertags-Spalten pro Jahr
yearly_holidays = df_dates.iloc[:, 1:].groupby(df_dates['Datum'].str[-4:]).sum()
print(yearly_holidays.sum(axis=1))

# 2. Liste aller Feiertage
print("\nAlle vorkommenden Feiertage:")
print(sorted(feiertags_reihenfolge))

# 3. Spezifische Prüfungen
print("\nNeujahr 2014-2017:")
neujahr_mask = df_dates['Datum'].str.contains('01.01.201[4-7]')
print(df_dates[neujahr_mask])

print("\nReformationstag 2017:")
reformationstag_mask = df_dates['Datum'].str.contains('31.10.2017')
print(df_dates[reformationstag_mask])

# 4. Prüfe auf Tage mit mehreren Feiertagen
print("\nTage mit mehreren Feiertagen:")
multiple_holidays = df_dates[df_dates.iloc[:, 1:].sum(axis=1) > 1]
print(multiple_holidays if not multiple_holidays.empty else "Keine doppelten Feiertage gefunden")

Anzahl Tage pro Jahr:
Datum
2013    184
2014    365
2015    365
2016    366
2017    365
2018    365
2019    365
dtype: int64

Anzahl Feiertage pro Jahr:
Datum
2013     3
2014    11
2015    11
2016    11
2017    12
2018    12
2019    12
dtype: int64

Alle vorkommenden Feiertage:
['Christi_Himmelfahrt', 'Erster_Weihnachtstag', 'Karfreitag', 'Neujahr', 'Ostermontag', 'Ostersonntag', 'Pfingstmontag', 'Pfingstsonntag', 'Reformationstag', 'Tag_der_Arbeit', 'Tag_der_Deutschen_Einheit', 'Zweiter_Weihnachtstag']

Neujahr 2014-2017:
           Datum  Neujahr  Karfreitag  Ostersonntag  Ostermontag  \
184   01.01.2014        1           0             0            0   
549   01.01.2015        1           0             0            0   
914   01.01.2016        1           0             0            0   
1280  01.01.2017        1           0             0            0   

      Tag_der_Arbeit  Christi_Himmelfahrt  Pfingstsonntag  Pfingstmontag  \
184                0                    0             

In [19]:
# df mit einer Spalte versehen die einfach True oder False enthält, wenn ein Feiertag ist
df_dates['Feiertag'] = df_dates.iloc[:, 1:].sum(axis=1) > 0
df_dates['Feiertag'] = df_dates['Feiertag'].astype(int)

#subset mit datum und feiertag
df_dates = df_dates[['Datum', 'Feiertag']]

#datum noch ins fomrat yyyy-mm-dd umwandeln
df_dates['Datum'] = pd.to_datetime(df_dates['Datum'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')
print(df_dates)

#exportieren
df_dates.to_csv('feiertage_final.csv', index=False, sep=';', encoding='utf-8')



           Datum  Feiertag
0     2013-07-01         0
1     2013-07-02         0
2     2013-07-03         0
3     2013-07-04         0
4     2013-07-05         0
...          ...       ...
2370  2019-12-27         0
2371  2019-12-28         0
2372  2019-12-29         0
2373  2019-12-30         0
2374  2019-12-31         0

[2375 rows x 2 columns]
